In [1]:
%run 'about_ui.ipynb'
%run 'process_ui.ipynb'

HTML(value='\n<style>\n.leaflet-pane {\n    z-index : 2 !important;\n}\n.leaflet-top, .leaflet-bottom {\n    z…

ResizeTrigger()

In [2]:
process_tile

DmpTile(align_center=True, children=[Card(children=[Html(children=['Damage Proxy Map - Inputs'], tag='h2'), Fl…

In [15]:
import gdal, os
from zipfile import ZipFile
from pathlib import Path

tmp_dir = Path('/ram')
# kmz and dmp output
# write a color file to tmp
ctfile = tmp_dir.joinpath('colourtable.txt')
f = open(ctfile, "w")
ct = ["0 0 0 0 0\n"
    "27 253 246 50 255\n"
    "35 253 169 50 255\n"
    "43 253 100 50 255\n"
    "51 253 50 50 255\n"
    "59 255 10 10 255\n"
    "255 253 0 0 255"
    ]
f.writelines(ct)
f.close()


out_dmp_tif = Path('/home/vollrath/test_dpm/Damage_Proxy_Map/dmp_A121_201029_201110_201122.tif')
#demopts = gdal.DEMProcessingOptions(colorFilename=str(ctfile), addAlpha=True)
#gdal.DEMProcessing(str(out_dmp_tif), str(out_ds_tif), 'color-relief', options=demopts)         

opts = gdal.TranslateOptions(format='KMLSUPEROVERLAY', creationOptions=["format=png"])
gdal.Translate(str(out_dmp_tif.with_suffix('.kmz')), str(out_dmp_tif), options=opts)

### adding legend like this to KMZ
added = [
    "\t\t<ScreenOverlay>\n",
    "\t\t\t<name>\n",
    "Legend: Damage Proxy Map\n",
    "\t\t\t</name>\n",
    "\t\t\t<Icon>\n",
    "\t\t\t\t<href>https://raw.githubusercontent.com/12rambau/damage_proxy_map/refactoring/component/message/legend.png</href>\n",
    "\t\t\t</Icon>\n",
    '\t\t\t<overlayXY x="0.98" y="0.14" xunits="fraction" yunits="fraction"/>\n',
    '\t\t\t<screenXY x="0.98" y="0.14" xunits="fraction" yunits="fraction"/>\n',
    '\t\t\t<rotationXY x="0.5" y="0.5" xunits="fraction" yunits="fraction"/>\n',
    '\t\t\t<size x="0.1" y="0.265" xunits="fraction" yunits="fraction"/>\n',
    "\t\t</ScreenOverlay>\n",
    "\t</Document>\n",
    "</kml>\n"   
]
tmpzip = tmp_dir.joinpath('zipped')
tmpzip.mkdir(parents=True, exist_ok=True)

with ZipFile(out_dmp_tif.with_suffix('.kmz')) as zip_ref:
    zip_ref.extractall(tmpzip)
    with open(tmpzip.joinpath('doc.kml')) as f:

        lines = f.readlines()
        lines = lines[:-2]
        lines.extend(added)

    with open(tmpzip.joinpath('doc.kml'), 'w') as f:
        for ele in lines:
            f.write(ele)

with ZipFile(out_dmp_tif.with_suffix('.kmz'), 'w') as zip_ref:
    # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk(tmpzip):
        print(folderName)
        for filename in filenames:
            #create complete filepath of file in directory
            filePath = os.path.join(folderName, filename)
            #print(filePath)
            # Add file to zip
            zip_ref.write(filePath, os.path.join('/0/0/', os.path.basename(filePath)))

/ram/zipped
/ram/zipped/0
/ram/zipped/0/0


In [16]:
!unzip '/home/vollrath/test_dpm/Damage_Proxy_Map/dmp_A121_201029_201110_201122.kmz' -d '/home/vollrath/kmz'

Archive:  /home/vollrath/test_dpm/Damage_Proxy_Map/dmp_A121_201029_201110_201122.kmz
 extracting: /home/vollrath/kmz/0/0/doc.kml  
 extracting: /home/vollrath/kmz/0/0/0.kml  
 extracting: /home/vollrath/kmz/0/0/0.png  
